In [218]:
import numpy as np
import pandas as pd
from pykalman import KalmanFilter
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

In [219]:
a= 2251262576%7
a+1

4

In [220]:
train_df=pd.read_csv('./playground-series-s5e4/train.csv')
test_df=pd.read_csv('./playground-series-s5e4/test.csv')


In [221]:
thursday_df=train_df[train_df['Publication_Day']=='Thursday']
thursday_df.head(10)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
7,7,News Roundup,Episode 44,48.52,News,44.99,Thursday,Night,20.12,0.0,Positive,22.37517
9,9,Music Matters,Episode 81,NaN,Music,82.18,Thursday,Night,59.72,3.0,Neutral,45.94761
10,10,Sports Central,Episode 66,106.41,Sports,84.29,Thursday,Evening,32.84,3.0,Positive,74.08016
15,15,Tech Talks,Episode 47,68.33,Technology,75.33,Thursday,Evening,92.90,0.0,Neutral,63.97718
17,17,Sport Spot,Episode 32,95.45,Sports,92.19,Thursday,Afternoon,70.29,2.0,Neutral,79.67749
38,38,Music Matters,Episode 32,86.84,Music,86.65,Thursday,Morning,28.27,1.0,Neutral,81.49839
40,40,Sports Central,Episode 77,69.66,Sports,57.32,Thursday,Evening,61.32,1.0,Neutral,50.05332
55,55,World Watch,Episode 6,34.32,News,20.33,Thursday,Morning,83.26,0.0,Negative,16.04940
57,57,Style Guide,Episode 23,107.61,Lifestyle,33.32,Thursday,Evening,96.94,0.0,Negative,77.01387


In [222]:
thursday_df.isnull().sum()


id                                 0
Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         12566
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    20443
Number_of_Ads                      0
Episode_Sentiment                  0
Listening_Time_minutes             0
dtype: int64

In [223]:
thursday_df.dtypes

id                               int64
Podcast_Name                    object
Episode_Title                   object
Episode_Length_minutes         float64
Genre                           object
Host_Popularity_percentage     float64
Publication_Day                 object
Publication_Time                object
Guest_Popularity_percentage    float64
Number_of_Ads                  float64
Episode_Sentiment               object
Listening_Time_minutes         float64
dtype: object

In [224]:
for col in thursday_df.select_dtypes(include='float64'):
    thursday_df[col]=thursday_df[col].fillna(thursday_df[col].mode()[0])
thursday_df.isnull().sum()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16912\4006160386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thursday_df[col]=thursday_df[col].fillna(thursday_df[col].mode()[0])


id                             0
Podcast_Name                   0
Episode_Title                  0
Episode_Length_minutes         0
Genre                          0
Host_Popularity_percentage     0
Publication_Day                0
Publication_Time               0
Guest_Popularity_percentage    0
Number_of_Ads                  0
Episode_Sentiment              0
Listening_Time_minutes         0
dtype: int64

In [225]:
le = LabelEncoder()
for col in thursday_df.columns:
    if thursday_df[col].dtype == 'object':
        le = LabelEncoder()
        thursday_df[col] = le.fit_transform(thursday_df[col].astype(str))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16912\218396458.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thursday_df[col] = le.fit_transform(thursday_df[col].astype(str))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16912\218396458.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thursday_df[col] = le.fit_transform(thursday_df[col].astype(str))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16912\218396458.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [226]:
# thursday_df.to_csv('thursday.csv', index=False, encoding='utf-8-sig')

In [227]:
df_sorted = thursday_df.sort_values(by='Publication_Time')
data = df_sorted['Listening_Time_minutes'].values


In [228]:
df_sorted.tail(10)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
372002,372002,15,12,43.16,1,33.12,0,3,13.90,0.0,2,41.38941
371980,371980,8,31,7.89,6,22.26,0,3,85.91,3.0,0,6.43583
749903,749903,6,32,6.44,9,94.83,0,3,8.35,1.0,0,0.42067
749870,749870,44,71,25.73,9,39.10,0,3,33.49,0.0,1,10.82000
749868,749868,34,79,30.69,9,33.15,0,3,27.19,0.0,2,89.95920
749858,749858,27,79,97.37,4,81.01,0,3,93.75,1.0,2,92.37949
202,202,30,92,49.81,5,93.69,0,3,56.43,1.0,2,41.18219
0,0,34,98,30.69,9,74.81,0,3,56.43,0.0,2,31.41998
172,172,26,99,30.69,2,79.73,0,3,73.54,1.0,2,54.52000
146,146,42,61,86.83,8,69.61,0,3,58.89,1.0,1,49.88000


In [229]:

# Mô hình 1: Kalman đơn giản
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=0,
    initial_state_covariance=1,
    observation_covariance=1,
    transition_covariance=0.01
)

# Lọc chuỗi quan sát
state_means, state_covs = kf.filter(data)

# Dự đoán n bước tiếp theo
n_steps = 10
last_state = state_means[-1]
last_cov = state_covs[-1]
predictions = []
for _ in range(n_steps):
    last_state = np.dot([[1]], last_state)
    last_cov = np.dot([[1]], np.dot(last_cov, [[1]])) + 0.01
    predicted = np.dot([[1]], last_state)
    predictions.append(predicted.item())

# Xuất ra file CSV
future = pd.DataFrame({
    'time_step': range(len(data), len(data) + n_steps),
    'observed': [np.nan] * n_steps,
    'predicted': predictions
})
future.to_csv('kalman_prediction.csv', index=False)


In [230]:

model = ARIMA(data, order=(1, 1, 1))  # AR(1), I(1), MA(1)
model_fit = model.fit()

# Bước 3: Dự báo n bước tiếp theo
n_steps = 10
forecast = model_fit.forecast(steps=n_steps)

# Bước 4: Lưu kết quả vào CSV
observed_series = pd.Series(data, name="observed")
predicted_series = pd.Series([np.nan]*len(data) + list(forecast), name="predicted")

result_df = pd.concat([observed_series, predicted_series], axis=1)
result_df.to_csv("arima_prediction.csv", index_label="time_step")

print("✅ Dự đoán ARIMA đã được lưu vào file: arima_prediction.csv")

✅ Dự đoán ARIMA đã được lưu vào file: arima_prediction.csv


In [ ]:
# # Mô hình 2: Kalman với xu hướng
kf2 = KalmanFilter(
    transition_matrices=[[1, 1], [0, 1]],
    observation_matrices=[[1, 0]],
    initial_state_mean=[data[0], 0],
    initial_state_covariance=np.eye(2),
    observation_covariance=1,
    transition_covariance=0.01 * np.eye(2)
)
state_means2, _ = kf2.filter(data)
n = 10  # Số bước dự đoán tiếp theo
last_state = state_means2[-1]  # Trạng thái cuối cùng
transition_matrix = np.array([[1, 1], [0, 1]])  # Ma trận chuyển trạng thái
observation_matrix = np.array([[1, 0]])  # Ma trận quan sát

predictions = []  # Danh sách chứa kết quả dự đoán
state = last_state.copy()  # Copy trạng thái cuối cùng

# Dự đoán n bước
for _ in range(n):
    # Dự đoán trạng thái tiếp theo
    state = transition_matrix @ state
    # Dự đoán giá trị (quan sát)
    predicted = observation_matrix @ state
    predictions.append(predicted[0])  # Lưu giá trị dự đoán
future = pd.DataFrame({
    'time_step': range(len(data), len(data) + n),
    'observed': [np.nan] * n,  # Không có quan sát mới
    'predicted': predictions  # Các giá trị dự đoán
})

# Ghi vào file CSV
future.to_csv('kalman2_prediction.csv', index=False)


In [232]:
# Mô hình 3: Kalman với xu hướng + nhiễu lớn
# kf3 = KalmanFilter(
#     transition_matrices=[[1, 1], [0, 1]],
#     observation_matrices=[[1, 0]],
#     initial_state_mean=[data[0], 0],
#     initial_state_covariance=np.eye(2),
#     observation_covariance=4,
#     transition_covariance=0.05 * np.eye(2)
# )
# state_means3, _ = kf3.filter(data)



In [233]:
# Vẽ biểu đồ so sánh
# plt.figure(figsize=(15, 6))
# plt.plot(data, color='gray', alpha=0.4, label='Observed')
# plt.plot(state_means1[:, 0], label='Model 1: Simple Kalman')
# plt.plot(state_means2[:, 0], label='Model 2: Kalman with Trend')
# plt.plot(state_means3[:, 0], label='Model 3: Trend + High Noise')
# plt.legend()
# plt.title('Kalman Filter Models on Listening_Time_minutes')
# plt.xlabel('Time Index (sorted by Publication_Time)')
# plt.ylabel('Listening Time (minutes)')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [234]:
# observations = data.values  # sử dụng toàn bộ cột quan sát nếu là dữ liệu chuỗi

# # Bước 2: Khởi tạo Kalman Filter
# kf = KalmanFilter(
#     transition_matrices=np.eye(observations.shape[1]),  # A
#     observation_matrices=np.eye(observations.shape[1]),  # C
#     initial_state_mean=np.zeros(observations.shape[1]),
#     transition_covariance=0.1 * np.eye(observations.shape[1]),  # Q
#     observation_covariance=0.1 * np.eye(observations.shape[1])  # R
# )

# # Bước 3: Áp dụng Kalman Filter
# state_means, state_covariances = kf.filter(observations)

# # Bước 4: Vẽ kết quả so sánh dữ liệu gốc và trạng thái ẩn được ước lượng
# plt.figure(figsize=(12, 6))
# for i in range(observations.shape[1]):
#     plt.plot(observations[:, i], label=f'Observation {i}')
#     plt.plot(state_means[:, i], '--', label=f'Estimated State {i}')
# plt.legend()
# plt.title('Kalman Filter - LDS Estimation')
# plt.show()

MÔ HÌNH ARIMA

In [235]:
# series = data['Publication_Time']
# model = SARIMAX(series, order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False)
# results = model.fit(disp=False)
# state_estimates = results.filtered_state
# state_estimates

In [236]:
# plt.figure(figsize=(12, 6))
# for i in range(state_estimates.shape[0]):
#     plt.plot(state_estimates[i], label=f'State {i}')
# plt.title('Estimated Hidden States from Kalman Filter')
# plt.xlabel('Time')
# plt.ylabel('State value')
# plt.legend()
# plt.show()

In [237]:
# forecast = results.get_prediction()
# predicted_mean = forecast.predicted_mean
# conf_int = forecast.conf_int()
# plt.figure(figsize=(12, 6))
# plt.plot(series.index, series, label='Observed')
# plt.plot(predicted_mean.index, predicted_mean, color='red', label='Kalman-smoothed ARIMA')
# plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
# plt.legend()
# plt.title('ARIMA + Kalman Filter Forecasting')
# plt.show()